[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sreejakr/openforest4d-forest-metrics/blob/main/notebooks/intersection_data_retriever.ipynb)

# Download EPT Tiles in Jupyter Notebook

This notebook automates the retrieval of lidar point‑cloud data for a specified region of interest by leveraging the USGS 3DEP Entwine Point Tile (EPT) service. It consists of two principal stages:

1. **Intersection Polygon Computation**  
   - Reads the master catalog of USGS 3DEP boundaries (EPSG:4326).  
   - Selects two collections of interest (for example, pre‑ and post‑disturbance surveys) by their `name` fields.  
   - Computes the geometric intersection of those two footprints, producing a single polygon that defines the exact overlap for which both datasets exist.  
   - Writes this polygon to `data/placer_intersection.geojson` so that it can be used again later.

2. **Buffered Tile Generation & EPT Download**  
   - Reprojects the intersection polygon to UTM Zone 10N (EPSG:32610) so that all distances and areas are in meters. This target CRS can be changed to any other UTM zone by updating the EPSG code to match the study area’s longitude band.  
   - Constructs a regular grid of square tiles (1000 m × 1000 m) covering the intersection, with a 20 m buffer around each tile to avoid missing edge points.  
   - Iterates over each buffered tile:  
     - Invokes PDAL’s EPT reader to request only the points falling within the buffered polygon.  
     - Transforms the returned points into the target CRS.  
     - Filters out empty tiles to minimize storage.  
     - Writes each non‑empty tile as a compressed `.laz` file named by its lower‑left coordinates (e.g., `500000_4200000.laz`) into `Placer_2012_Tiled` (and a matching folder for the second collection).

---

## Motivation and Benefits

- By computing the exact overlap of two surveys, analysis focuses only on areas where comparable data exist, eliminating gaps and reducing bias in comparisons.  
- EPT queries return only the points within each buffered tile rather than downloading entire collections, greatly reducing bandwidth and local storage requirements.  
- Breaking the intersection into buffered tiles enables parallel or distributed processing of large regions, accommodating very high‑resolution lidar datasets without memory exhaustion.  
- All steps, from boundary loading through tile generation and download—are fully scripted, ensuring that any researcher can reproduce the same output simply by rerunning the notebook with identical parameters.  
- The resulting `.laz` tiles feed directly into subsequent reprojection, tiling, metric extraction, and differencing workflows, forming the foundation of the OpenForest4D change‑detection pipeline.  


---

##  Install Dependencies

```bash
# If needed, uncomment and run:
# !pip install geopandas shapely pyproj pdal
```

##  User Configuration

Edit these variables to match your environment:

#### 1. Input GeoJSON with USGS 3DEP boundaries (EPSG:4326)
- **Variable:** `boundaries_geojson`  
- **Description:** Local path to the GeoJSON containing all 3DEP dataset footprints.  
- **Note:**  The file has been included in `data/usgs_3dep_boundaries.geojson` (EPSG:4326).
    Or get the latest from:    https://github.com/OpenTopography/Data_Catalog_Spatial_Boundaries/blob/main/usgs_3dep_boundaries.geojson

#### 2. Feature Names to Intersect
- **Variables:** `name_a`, `name_b`  
- **Description:** Exact strings matching the `"name"` property in your GeoJSON. The script will extract only these two footprints and compute their geometric overlap.

#### 3. Intersection Output Path
- **Variable:** `intersection_geojson`  
- **Description:** File path where the resulting intersection polygon GeoJSON will be saved. Load this in QGIS or feed it downstream.

#### 4. EPT Endpoints for Each Dataset
- **Variables:** `ept_path_a`, `ept_path_b`  
- **Description:** The Entwine Point Tile (EPT) URLs for each 3DEP dataset. Prefix with `ept://` so PDAL can read them directly.  
- **How to obtain:**  
  1. Browse the AWS Public lidar S3 bucket:  
     ```
     https://s3-us-west-2.amazonaws.com/usgs-lidar-public/
     ```  
  2. Find your dataset folder (e.g. `CA_PlacerCo_2012`) and point PDAL at its `ept.json` URL:  
     ```
     ept://https://s3-us-west-2.amazonaws.com/usgs-lidar-public/CA_PlacerCo_2012/ept.json
     ```  
  3. Use that string in your `ept_path_*` variable.

#### 5. Output Directories for Tiles
- **Variables:** `output_dir_a`, `output_dir_b`  
- **Description:** Folders where PDAL will write each tiled `.las`/`.laz`. The script will create these directories if they don’t already exist. This is where the laz files get stored for each of the years based on the intersecting data.

#### 6. Tiling & Reprojection Parameters
- **Variables:**  
  - `tile_size` – side length of each square tile (in meters).  
  - `buffer_size` – extra margin around each tile (in meters) to guarantee overlap.  
  - `target_epsg` – output coordinate system code (e.g. `"EPSG:32610"` for UTM Zone 10 N).  
- **Adjust as needed** for your study area’s scale, overlap tolerance, and desired CRS.



In [ ]:
import sys

# Connect to local google drive when opened in colab
if "google.colab" in sys.modules:
    from google.colab import drive
    drive.mount("/gdrive/")

In [ ]:
# Input GeoJSON with USGS 3DEP boundaries (EPSG:4326)
boundaries_geojson = r"C:/Users/sreeja/Documents/OpenForest4D/usgs_3dep_boundaries.geojson"

# Feature names matching the 'name' column in the GeoJSON
name_a = "CA PlacerCo 2012"
name_b = "USGS LPC CA NoCAL Wildfires B5a 2018"

# Path to save the computed intersection
intersection_geojson = r"C:/Users/sreeja/Documents/CA_Placer_Co/sc_intersection.geojson"

# EPT endpoints for each dataset
ept_path_a = "ept://https://s3-us-west-2.amazonaws.com/usgs-lidar-public/CA_PlacerCo_2012"
ept_path_b = "ept://https://s3-us-west-2.amazonaws.com/usgs-lidar-public/USGS_LPC_CA_NoCAL_Wildfires_B5a_2018"

# Output directories for tiles
output_dir_a = r"C:/Users/sreeja/Documents/CA_Placer_Co/Placer_2012_Tiled"
output_dir_b = r"C:/Users/sreeja/Documents/CA_Placer_Co/Placer_2018_Tiled"

# Tiling parameters and target CRS
tile_size = 1000      # meters
buffer_size = 20      # meters
target_epsg = "EPSG:32610"  # UTM Zone 10N

##  3. Imports & Function Definitions


In [10]:
import os
import json
import time
import geopandas as gpd
from shapely.geometry import box
from pyproj import Transformer
import pdal

# Function to compute intersection of two named features

In [11]:
def compute_intersection(geojson_path: str, feature1: str, feature2: str, out_path: str) -> tuple:
    """
    Read a boundary GeoJSON, select two named features, compute their spatial intersection,
    save it to a new GeoJSON, and return the intersection bounding box.

    Args:
        geojson_path: Path to input GeoJSON (EPSG:4326).
        feature1: Exact name of the first feature.
        feature2: Exact name of the second feature.
        out_path: Path to save the intersection GeoJSON.

    Returns:
        Tuple (xmin, ymin, xmax, ymax) in EPSG:4326.
    """
    gdf = gpd.read_file(geojson_path)
    a = gdf[gdf['name'] == feature1]
    b = gdf[gdf['name'] == feature2]
    if a.empty or b.empty:
        raise ValueError(f"Features not found: {feature1}, {feature2}")

    if a.crs != b.crs:
        b = b.to_crs(a.crs)

    inter = gpd.overlay(a, b, how='intersection')
    if inter.empty:
        raise RuntimeError("No spatial overlap between features.")

    inter.to_file(out_path, driver='GeoJSON')
    print(f"Intersection saved to: {out_path}")
    return tuple(inter.total_bounds)

# Function to tile and download EPT lidar

The download_ept_tiles function encapsulates a complete workflow for chopping up a large lidar data footprint into manageable tiles and downloading just the points that are needed, reprojecting and saving each tile as a compressed LAS/LAZ. Here’s how it works, step by step:

1. **Preparing the Output Folder**
As soon a the function is called, it ensures that your designated output directory exists by calling os.makedirs(output_dir, exist_ok=True). This means you don’t have to worry about creating folders yourself, if it isn’t there, Python will make it for you.

2. **Reprojecting Your Geographic Bounds**
The function takes your WGS84 "bounds" (a 4‑tuple of longitudes and latitudes in EPSG:4326) and uses a PROJ transformer (via pyproj.Transformer) to convert those corner coordinates into your target_epsg projection (for example, UTM Zone 10N). All downstream tiling happens in this projected CRS so that distances (tile sizes and buffers) in meters map directly to the grid.

3. **Snapping to a Regular Tile Grid**
Once your minimum and maximum X/Y in the target CRS are known, the code snaps those values to the nearest multiple of tile_size. For example, if your data spans from 512 m to 3852 m eastings, and you use 1000 m tiles, the loop will start at 0 m and run tiles at [0–1000], [1000–2000], [2000–3000], and [3000–4000 m]. This guarantees that every tile is the same size and that the full extent CAN BE COVERES (plus a tile’s worth of extra margin at the edges).

4. **Looping over Every Tile**
Two nested while loops iterate over every grid cell:
 - The outer loop increments Y by tile_size
 - The inner loop increments X by tile_size
For each cell, it builds a unique tile_id string like "3000_2000" representing the lower‑left corner of that square.

5. **Skipping Already-Downloaded Tiles**
Before doing any heavy work, the function checks if tile_id.laz already exists in your output directory. If so, it prints a "Skipping existing tile" message and moves on. This lets you safely re‑run the notebook without re‑downloading data you already have.

6. **Applying a Buffer**
Each tile is optionally expanded by buffer meters on all sides (so a 1040 m square is requested if a 20 m buffer is used for a 1000 m tile). Buffers are crucial to avoid edge artifacts when interpolating or merging later on.

7. **Building a PDAL Pipeline**
For each tile, the function builds a small JSON‐style PDAL pipeline:

 - `readers.ept` points at your EPT URL and restricts the request to the buffered bounds.
 - `filters.reprojection` immediately reprojects those points into your target_epsg CRS.
 - `writers.las` writes out a compressed LAS/LAZ file, named by the tile_id.

8. **Executing & Checking Point Counts**
The PDAL pipeline is executed via p.execute(). That returns the number of points read for that tile.
 - If count > 0, the file stays on disk and you see a "Downloaded X points" message.
 - If count == 0, the code removes the empty file (so thousands of zero‑point tiles are not accumulated) and logs "No points for tile …; file skipped."

9. * Timing & Reporting**
Finally, the function keeps track of total elapsed time across all tiles (with time.time() before and after the loops) and prints a summary like
`Completed downloading tiles to C:/…/Placer_2012_Tiled in 123.4 seconds`.

##### Why each step matters

 - Regular tiling ensures that your downstream analyses (DEM generation, classification, etc.) can be parallelized and that no tile is too large for memory.
 - Buffers prevent edge‐of‐tile artifacts when filters look at neighboring points.
 - Reprojection guarantees that all point files share the same coordinate system and spatial resolution.
 - Compressed output (LAZ vs. LAS) saves disk space and I/O time.

In [12]:
def download_ept_tiles(ept_url: str, output_dir: str, bounds: tuple,
                       tile_size: int, buffer: int, target_epsg: str):
    """
    Tile a WGS84 bounding box, download lidar from an EPT endpoint for each tile,
    reproject to the target CRS, and save as compressed .laz files, skipping any
    tiles that contain zero points.

    Args:
        ept_url: EPT endpoint URL.
        output_dir: Directory to save the output tiles.
        bounds: Tuple (xmin, ymin, xmax, ymax) in EPSG:4326.
        tile_size: Tile dimension in meters.
        buffer: Buffer around each tile in meters.
        target_epsg: Output projection (e.g., 'EPSG:32610').
    """
    os.makedirs(output_dir, exist_ok=True)

    # Project the WGS84 bounds into the target CRS
    transformer = Transformer.from_crs('EPSG:4326', target_epsg, always_xy=True)
    x0, y0 = transformer.transform(bounds[0], bounds[1])
    x1, y1 = transformer.transform(bounds[2], bounds[3])

    # Snap to the tile grid
    min_x = int(x0 // tile_size) * tile_size
    min_y = int(y0 // tile_size) * tile_size
    max_x = int(x1 // tile_size + 1) * tile_size
    max_y = int(y1 // tile_size + 1) * tile_size

    print(f"Downloading tiles from X:{min_x}-{max_x}, Y:{min_y}-{max_y} in {target_epsg}")

    start = time.time()
    y = min_y
    while y < max_y:
        x = min_x
        while x < max_x:
            tile_id = f"{x}_{y}"
            out_file = os.path.join(output_dir, f"{tile_id}.laz")

            if os.path.isfile(out_file):
                print(f"Skipping existing tile: {tile_id}")
                x += tile_size
                continue

            # Build the buffered bounds string for PDAL
            bx0, bx1 = x - buffer, x + tile_size + buffer
            by0, by1 = y - buffer, y + tile_size + buffer
            bounds_str = f"([{bx0},{bx1}], [{by0},{by1}])/{target_epsg}"

            pipeline = {
                'pipeline': [
                    {'type': 'readers.ept',        'filename': ept_url, 'bounds': bounds_str},
                    {'type': 'filters.reprojection', 'out_srs': target_epsg},
                    {'type': 'writers.las',        'filename': out_file, 'compression': 'laszip'}
                ]
            }

            try:
                p = pdal.Pipeline(json.dumps(pipeline))
                count = p.execute()
                if count > 0:
                    print(f"Downloaded {count} points for tile {tile_id}")
                else:
                    # Remove the empty file and skip
                    if os.path.exists(out_file):
                        os.remove(out_file)
                    print(f"No points for tile {tile_id}; file skipped.")
            except Exception as e:
                print(f"Error on tile {tile_id}: {e}")

            x += tile_size
        y += tile_size

    elapsed = time.time() - start
    print(f"Completed downloading tiles to {output_dir} in {elapsed:.1f} seconds")


In [16]:
# Compute the intersection polygon in EPSG:4326
bounds_4326 = compute_intersection(
    boundaries_geojson, name_a, name_b, intersection_geojson
)

# # Download & tile the EPT point cloud for Dataset A
download_ept_tiles(ept_path_a, output_dir_a, bounds_4326, tile_size, buffer_size, target_epsg)

#Repeat for Dataset B
download_ept_tiles(ept_path_b, output_dir_b, bounds_4326, tile_size, buffer_size, target_epsg)